# Air quality analysis in Barcelona

## 1. Data import and cleaning

In [1]:
## import libraries

import pandas as pd 
import sqlalchemy 
import datetime 

In [2]:
raw_data = pd.read_csv('air_quality_Nov2017.csv')
station_links = pd.read_csv('air_stations_Nov2017.csv')

In [3]:
raw_data.head()

,Station,Air Quality,Longitude,Latitude,O3 Hour,O3 Quality,O3 Value,NO2 Hour,NO2 Quality,NO2 Value,PM10 Hour,PM10 Quality,PM10 Value,Generated,Date Time
0,Barcelona - Sants,Good,2.1331,41.3788,NaN,NaN,NaN,0h,Good,84.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104
1,Barcelona - Eixample,Moderate,2.1538,41.3853,0h,Good,1.0,0h,Moderate,113.0,0h,Good,36.0,01/11/2018 0:00,1541027104
2,Barcelona - Gràcia,Good,2.1534,41.3987,0h,Good,10.0,0h,Good,73.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104
3,Barcelona - Ciutadella,Good,2.1874,41.3864,0h,Good,2.0,0h,Good,86.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104
4,Barcelona - Vall Hebron,Good,2.1480,41.4261,0h,Good,7.0,0h,Good,69.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104


We remove the columns that are not needed for our analysis. In this case, the columns that are needed are: 
- Station
- Air Quality 
- O3 Quality
- 03 Value
- NO2 Quality
- NO2 Value
- PM10 Quality
- PM10 Value 
- Generated 
- Date Time 

In [4]:
print(raw_data.columns.tolist())

['Station', 'Air Quality', 'Longitude', 'Latitude', 'O3 Hour', 'O3 Quality', 'O3 Value', 'NO2 Hour', 'NO2 Quality', 'NO2 Value', 'PM10 Hour', 'PM10 Quality', 'PM10 Value', 'Generated', 'Date Time']


In [5]:
raw_data_selected = raw_data.copy()

raw_data_selected = raw_data[['Station', 'Air Quality', 'O3 Quality', 'O3 Value', 'NO2 Quality', 'NO2 Value', 'PM10 Quality', 'PM10 Value', 'Generated']]

### Analyzing the type of each attribute of the dataset

In [6]:

raw_data_selected.dtypes

Station          object
Air Quality      object
O3 Quality       object
O3 Value        float64
NO2 Quality      object
NO2 Value       float64
PM10 Quality     object
PM10 Value      float64
Generated        object
dtype: object

### Missing values

There are several ways to deal with missing values, in this case NaN values.
First of all, we need to know how many NaN values we have in our dataset. 

In [7]:
raw_data_selected.isna().sum()

Station            0
Air Quality        0
O3 Quality      1476
O3 Value        1643
NO2 Quality       55
NO2 Value        284
PM10 Quality    2022
PM10 Value      2097
Generated          0
dtype: int64

#### Method 1: dropping all the NaN values of the entire dataset. 

In this case we have removed all the rows that have a "NaN" value there. If we perform an analysis of what we have done, we can see the total amount of data we have lost 

In [8]:
before_rows = raw_data_selected.shape[0]
print(f"Number of rows before dropping all the NaN values: {before_rows}")

Number of rows before dropping all the NaN values: 5744


In [9]:
clean_air_quality =  raw_data_selected.copy()
clean_air_quality = raw_data_selected.dropna()
clean_air_quality.head()

,Station,Air Quality,O3 Quality,O3 Value,NO2 Quality,NO2 Value,PM10 Quality,PM10 Value,Generated
1,Barcelona - Eixample,Moderate,Good,1.0,Moderate,113.0,Good,36.0,01/11/2018 0:00
5,Barcelona - Palau Reial,Good,Good,11.0,Good,57.0,Good,23.0,01/11/2018 0:00
7,Barcelona - Observ Fabra,Good,Good,58.0,Good,3.0,Good,25.0,01/11/2018 0:00
9,Barcelona - Eixample,Good,Good,6.0,Good,80.0,Good,35.0,01/11/2018 1:00
13,Barcelona - Palau Reial,Good,Good,27.0,Good,38.0,Good,24.0,01/11/2018 1:00


In [10]:
after_rows = clean_air_quality.shape[0]
print (f"Number of rows after dropping all the NaN values: {after_rows}")

Number of rows after dropping all the NaN values: 2853


In [11]:
data_lost_perc = ((before_rows - after_rows)/before_rows)*100
print(f"The percentage of data that is NaN and has been removed is {data_lost_perc}")


The percentage of data that is NaN and has been removed is 50.330779944289695


#### Method 2: Instead of NaN removal, values interpolation 

#### Method 3: Instead of NaN removal, choose the last value of each station 

In [12]:
#time_index_air_quality = pd.DatetimeIndex(raw_data_selected["Generated"])

clean_air_quality.dtypes

#raw_data_selected.head()


Station          object
Air Quality      object
O3 Quality       object
O3 Value        float64
NO2 Quality      object
NO2 Value       float64
PM10 Quality     object
PM10 Value      float64
Generated        object
dtype: object

Since the time of the Date Time column is an object, it should be converted into a DateTime object in order to be able to convert it as the index of our databases

In [13]:


clean_air_quality['date_corrected'] = pd.to_datetime(clean_air_quality["Generated"])

clean_air_quality.set_index("date_corrected", inplace=True) # Inplace reassign dataset to indexed dataset. Is as if dataset_NEW=dataset_OLD

clean_air_quality.index


/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


DatetimeIndex(['2018-01-11 00:00:00', '2018-01-11 00:00:00',
               '2018-01-11 00:00:00', '2018-01-11 01:00:00',
               '2018-01-11 01:00:00', '2018-01-11 01:00:00',
               '2018-01-11 02:00:00', '2018-01-11 02:00:00',
               '2018-01-11 02:00:00', '2018-01-11 03:00:00',
               ...
               '2018-11-30 21:00:00', '2018-11-30 21:00:00',
               '2018-11-30 22:00:00', '2018-11-30 22:00:00',
               '2018-11-30 22:00:00', '2018-11-30 22:00:00',
               '2018-11-30 23:00:00', '2018-11-30 23:00:00',
               '2018-11-30 23:00:00', '2018-11-30 23:00:00'],
              dtype='datetime64[ns]', name='date_corrected', length=2853, freq=None)

In [14]:
clean_air_quality = clean_air_quality.drop("Generated", axis=1)

In [15]:
clean_air_quality.head()

,Station,Air Quality,O3 Quality,O3 Value,NO2 Quality,NO2 Value,PM10 Quality,PM10 Value
date_corrected,,,,,,,,
2018-01-11 00:00:00,Barcelona - Eixample,Moderate,Good,1.0,Moderate,113.0,Good,36.0
2018-01-11 00:00:00,Barcelona - Palau Reial,Good,Good,11.0,Good,57.0,Good,23.0
2018-01-11 00:00:00,Barcelona - Observ Fabra,Good,Good,58.0,Good,3.0,Good,25.0
2018-01-11 01:00:00,Barcelona - Eixample,Good,Good,6.0,Good,80.0,Good,35.0
2018-01-11 01:00:00,Barcelona - Palau Reial,Good,Good,27.0,Good,38.0,Good,24.0


Data description in Barcelona in November 2018

In [16]:
clean_air_quality.describe()

,O3 Value,NO2 Value,PM10 Value
count,2853.000000,2853.000000,2853.000000
mean,36.539432,32.680687,15.599369
std,23.707684,24.343297,7.820458
min,1.000000,1.000000,2.000000
25%,16.000000,12.000000,10.000000
50%,37.000000,26.000000,14.000000
75%,56.000000,50.000000,20.000000
max,100.000000,117.000000,44.000000


### Data splitting per station

First of all, we need to see which are the columns we have in our cleaned dataset to see what are the elements that we will split per station

In [17]:
print(clean_air_quality.columns.to_list())

['Station', 'Air Quality', 'O3 Quality', 'O3 Value', 'NO2 Quality', 'NO2 Value', 'PM10 Quality', 'PM10 Value']


Stations we have in Barcelona: 
1. Barcelona - Ciutadella 
2. Barcelona - Eixample
3. Barcelona - Gràcia 
4. Barcelona - Palau Reial
5. Barcelona - Poblenou
6. Barcelona - Sants
7. Barcelona - Vall Hebron
8. Barcelona - Vallvidrera, el Tibidabo i les Planes

### BCN- Ciutadella 

In [18]:
BCN_Ciutadella = clean_air_quality[clean_air_quality["Station"]=="Barcelona - Ciutadella"]

BCN_Ciutadella.head()

,Station,Air Quality,O3 Quality,O3 Value,NO2 Quality,NO2 Value,PM10 Quality,PM10 Value
date_corrected,,,,,,,,


### BCN-Eixample

In [19]:
#BCN_Sants = clean_air_quality[clean_air_quality["Station"] == "Barcelona - Sants"]

#BCN_Sants

#BCN_Eixample = 

#clean_air_quality.head()

BCN_Eixample = clean_air_quality[clean_air_quality["Station"]=="Barcelona - Eixample"]

BCN_Eixample.head()

,Station,Air Quality,O3 Quality,O3 Value,NO2 Quality,NO2 Value,PM10 Quality,PM10 Value
date_corrected,,,,,,,,
2018-01-11 00:00:00,Barcelona - Eixample,Moderate,Good,1.0,Moderate,113.0,Good,36.0
2018-01-11 01:00:00,Barcelona - Eixample,Good,Good,6.0,Good,80.0,Good,35.0
2018-01-11 02:00:00,Barcelona - Eixample,Good,Good,14.0,Good,64.0,Good,34.0
2018-01-11 03:00:00,Barcelona - Eixample,Good,Good,32.0,Good,44.0,Good,35.0
2018-01-11 04:00:00,Barcelona - Eixample,Good,Good,43.0,Good,34.0,Good,34.0


## 2. Database connection

In [20]:
## Import libraries

from sqlalchemy import create_engine


In [21]:
driver = 'mysql+pymysql:'
user = 'owner'
password = 'ir0n-h4ck'
ip = '104.197.101.244'
database = 'project-week-2-barcelona'


In [22]:
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'

In [23]:
engine = create_engine(connection_string)

In [24]:
#query = """
 #       SELECT * FROM authors
#"""

#query = """
#        CREATE TABLE kerim_table
#        SELECT * FROM authors
#"""
#df_db = pd.read_sql(query, engine)

In [25]:
#df_db = pd.read_sql(query, engine)

#print(query)


